In [9]:
# import dependencies

import pandas as pd
import datetime as dt
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine 

### Build our data dictionary. Data collected from IMDb website 100 Most Beautiful People on the Planet
### and 100 Ugly Celebrities.
### Other statistics were collected from taddlr and healthyceleb web sites

### 100 Most Beautiful People on the Planet: https://www.imdb.com/list/ls066164634/
### 100 Ugly Celebrities: https://www.imdb.com/list/ls064405717/
### https://healthyceleb.com
### https://taddlr.com/

In [10]:
# Data dictionary
celeb_dict = [
    {
        'Name': 'Angelina Jolie',
        'Sex': 'Female',
        'DOB': '06/04/1975',
        'Birth Year': 1975,
        'Eye Color': 'Gray',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.50,
        'Weight(lbs)': 119.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Johnny Depp',
        'Sex': 'Male',
        'DOB': '06/09/1963',
        'Birth Year': 1963,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Cheekbones',
        'Height(ft)': 5.83,
        'Weight(lbs)': 171.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Charlize Theron',
        'Sex': 'Female',
        'DOB': '08/07/1975', 
        'Birth Year': 1975,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Attractive',
        'Height(ft)': 5.75,
        'Weight(lbs)': 121.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Brad Pitt',
        'Sex': 'Male',
        'DOB': '12/18/1963',
        'Birth Year': 1963,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Jaw',
        'Height(ft)': 5.91,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Amber Heard',
        'Sex': 'Female',
        'DOB': '04/22/1986',
        'Birth Year': 1986,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Body',
        'Height(ft)': 5.58,
        'Weight(lbs)': 137.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Jared Leto',
        'Sex': 'Male',
        'DOB': '12/26/1971',
        'Birth Year': 1971,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.75,
        'Weight(lbs)': 152.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Natalie Portman',
        'Sex': 'Female',
        'DOB': '06/09/1981',
        'Birth Year': 1981,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Moles',
        'Height(ft)': 5.25,
        'Weight(lbs)': 110.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Channing Tatum',
        'Sex': 'Male',
        'DOB': '04/26/1980',
        'Birth Year': 1980,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.08, 
        'Weight(lbs)': 196.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Mila Kunis',
        'Sex': 'Female',
        'DOB': '08/14/1983',
        'Birth Year': 1983,
        'Eye Color': 'Green and Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Body',
        'Height(ft)': 5.33,
        'Weight(lbs)': 115.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Chris Hemsworth',
        'Sex': 'Male',
        'DOB': '08/11/1983',
        'Birth Year': 1983,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Voice',
        'Height(ft)': 6.25,
        'Weight(lbs)': 201.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Scarlett Johansson',
        'Sex': 'Female',
        'DOB': '11/22/1984',
        'Birth Year': 1984,
        'Eye Color': 'Green',
        'Hair Color': 'Brunette',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.25,
        'Weight(lbs)': 125.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Matt Bomer',
        'Sex': 'Male',
        'DOB': '10/11/1977',
        'Birth Year': 1977,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.91,
        'Weight(lbs)': 176.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Henry Cavill',
        'Sex': 'Male',
        'DOB': '05/05/1983',
        'Birth Year': 1983,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Jawline',
        'Height(ft)': 6.08,
        'Weight(lbs)': 203.0, 
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Gal Gadot',
        'Sex': 'Female',
        'DOB': '04/30/1985',
        'Birth Year': 1985,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Voice',
        'Height(ft)': 5.83,
        'Weight(lbs)': 141.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Leonardo DiCaprio',
        'Sex': 'Male',
        'DOB': '11/11/1974',
        'Birth Year': 1974,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Style',
        'Height(ft)': 6.00,
        'Weight(lbs)': 192.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Emilia Clarke',
        'Sex': 'Female',
        'DOB': '10/23/1986',
        'Birth Year': 1986,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.16,
        'Weight(lbs)': 119.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Jude Law',
        'Sex': 'Male',
        'DOB': '12/29/1972',
        'Birth Year': 1972,
        'Eye Color': 'Green',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Voice',
        'Height(ft)': 5.91,
        'Weight(lbs)': 192.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Kate Beckinsale',
        'Sex': 'Female',
        'DOB': '07/26/1973',
        'Birth Year': 1973,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Body',
        'Height(ft)': 5.58, 
        'Weight(lbs)': 119.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Michael Ealy',
        'Sex': 'Male',
        'DOB': '08/03/1973', 
        'Birth Year': 1973,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Cheekbones',
        'Height(ft)': 5.83,
        'Weight(lbs)': 167.5,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Meagan Good',
        'Sex': 'Female',
        'DOB': '08/08/1981',
        'Birth Year': 1981,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Eyebrows',
        'Height(ft)': 5.41,
        'Weight(lbs)': 126.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Ryan Reynolds',
        'Sex': 'Male',
        'DOB': '10/23/1976',
        'Birth Year': 1976,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 6.16,
        'Weight(lbs)': 185.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Lucy Hale',
        'Sex': 'Female',
        'DOB': '06/14/1989',
        'Birth Year': 1989,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.16,
        'Weight(lbs)': 108.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Zac Efron',
        'Sex': 'Male',
        'DOB': '10/18/1987',
        'Birth Year': 1987,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.66,
        'Weight(lbs)': 181.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Adelaide Kane',
        'Sex': 'Female',
        'DOB': '08/09/1990',
        'Birth Year': 1990,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Body',
        'Height(ft)': 5.33,
        'Weight(lbs)': 110.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Chris Pratt',
        'Sex': 'Male',
        'DOB': '06/21/1979',
        'Birth Year': 1979,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Height',
        'Height(ft)': 6.16,
        'Weight(lbs)': 223.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Kerry Washington',
        'Sex': 'Female',
        'DOB': '01/31/1977',
        'Birth Year': 1977,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.33,
        'Weight(lbs)': 117.0,
        'Zodiac Sign': 'Aquarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Jensen Ackles',
        'Sex': 'Male',
        'DOB': '03/01/1978',
        'Birth Year': 1978,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.08,
        'Weight(lbs)': 183.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Shay Mitchell ',
        'Sex': 'Female',
        'DOB': '04/10/1987',
        'Birth Year': 1987,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.58,
        'Weight(lbs)': 132.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Ian Somerhalder',
        'Sex': 'Male',
        'DOB': '12/08/1978',
        'Birth Year': 1978,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.75,
        'Weight(lbs)': 165.5,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Alycia Debnam-Carey',
        'Sex': 'Female',
        'DOB': '07/20/1993',
        'Birth Year': 1993,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.58,
        'Weight(lbs)': 121.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Ryan Gosling',
        'Sex': 'Male',
        'DOB': '11/12/1980',
        'Birth Year': 1980,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Jawline',
        'Height(ft)': 6.00,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Ruby Rose',
        'Sex': 'Female',
        'DOB': '03/20/1986',
        'Birth Year': 1986,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.58,
        'Weight(lbs)': 128.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Ashton Kutcher',
        'Sex': 'Male',
        'DOB': '02/07/1978',
        'Birth Year': 1978,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Jawline',
        'Height(ft)': 6.16, 
        'Weight(lbs)': 176.0,
        'Zodiac Sign': 'Aquarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Kate Upton',
        'Sex': 'Female',
        'DOB': '07/10/1992',
        'Birth Year': 1992,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.83,
        'Weight(lbs)': 156.5,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Ricky Whittle',
        'Sex': 'Male',
        'DOB': '12/31/1981',
        'Birth Year': 1981,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 6.08,
        'Weight(lbs)': 200.5,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Jessica Biel',
        'Sex': 'Female',
        'DOB': '03/03/1982',
        'Birth Year': 1982,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.58,
        'Weight(lbs)': 121.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Jason Momoa',
        'Sex': 'Male',
        'DOB': '08/01/1979',
        'Birth Year': 1979,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 6.33,
        'Weight(lbs)': 234.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Marie Avgeropoulos',
        'Sex': 'Female',
        'DOB': '06/17/1986',
        'Birth Year': 1986,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.41,
        'Weight(lbs)': 121.0, 
        'Zodiac Sign': ' Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Stephen Amell',
        'Sex': 'Male',
        'DOB': '05/08/1981',
        'Birth Year': 1981,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 6.08,
        'Weight(lbs)': 182.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Kate Hudson',
        'Sex': 'Female',
        'DOB': '04/19/1979',
        'Birth Year': 1979,
        'Eye Color': 'Green',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.50,
        'Weight(lbs)': 121.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Chace Crawford',
        'Sex': 'Male',
        'DOB': '07/18/1985',
        'Birth Year': 1985,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.00,
        'Weight(lbs)': 175.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Olivia Wilde',
        'Sex': 'Female',
        'DOB': '03/10/1984',
        'Birth Year': 1984,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.58,
        'Weight(lbs)': 115.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Michael Fassbender',
        'Sex': 'Male',
        'DOB': '04/02/1977',
        'Birth Year': 1977,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Jawline',
        'Height(ft)': 6.00, 
        'Weight(lbs)': 174.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Daisy Ridley',
        'Sex': 'Female',
        'DOB': '04/10/1992',
        'Birth Year': 1992,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.58,
        'Weight(lbs)': 121.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Chris Pine',
        'Sex': 'Male',
        'DOB': '08/26/1980',
        'Birth Year': 1980,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.00,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Cate Blanchett',
        'Sex': 'Female',
        'DOB': '05/14/1969',
        'Birth Year': 1969,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Cheekbones',
        'Height(ft)': 5.66,
        'Weight(lbs)': 130.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Gerard Butler',
        'Sex': 'Male',
        'DOB': '11/13/1969',
        'Birth Year': 1969,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 6.16,
        'Weight(lbs)': 187.5,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Emma Watson',
        'Sex': 'Female',
        'DOB': '04/15/1990',
        'Birth Year': 1990,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes', 
        'Height(ft)': 5.41,
        'Weight(lbs)': 117.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Luke Evans',
        'Sex': 'Male',
        'DOB': '04/15/1979',
        'Birth Year': 1979,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 6.00,
        'Weight(lbs)': 174.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Penélope Cruz',
        'Sex': 'Female',
        'DOB': '04/28/1974',
        'Birth Year': 1974,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.50,
        'Weight(lbs)': 137.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Jamie Dornan',
        'Sex': 'Male',
        'DOB': '05/01/1982',
        'Birth Year': 1982,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.00,
        'Weight(lbs)': 170.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Keira Knightley',
        'Sex': 'Female',
        'DOB': '03/26/1985',
        'Birth Year': 1985,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.58,
        'Weight(lbs)': 119.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Chris Evans',
        'Sex': 'Male',
        'DOB': '06/13/1981', 
        'Birth Year': 1981,
        'Eye Color':'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 6.00,
        'Weight(lbs)': 176.5,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Felicity Jones',
        'Sex': 'Female',
        'DOB': '10/17/1983',
        'Birth Year': 1983,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 5.25,
        'Weight(lbs)': 121,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Jonathan Rhys Meyers',
        'Sex': 'Male',
        'DOB': '07/27/1977',
        'Birth Year': 1977,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.75,
        'Weight(lbs)': 159.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Megan Fox',
        'Sex': 'Female',
        'DOB': '05/16/1986',
        'Birth Year': 1986,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.33,
        'Weight(lbs)': 114.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Alexander Dreymon',
        'Sex': 'Male',
        'DOB': '02/07/1983',
        'Birth Year': 1983,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.83,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Aquarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Monroe Hayden',
        'Sex': 'Female',
        'DOB': '11/08/1988',
        'Birth Year': 1988,
        'Eye Color': 'Gray',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.33, 
        'Weight(lbs)': 114.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Douglas Booth',
        'Sex': 'Male',
        'DOB': '07/09/1992',
        'Birth Year': 1992,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Jawline',
        'Height(ft)': 6.08,
        'Weight(lbs)': 170.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Kristin Kreuk',
        'Sex': 'Female',
        'DOB': '12/30/1982',
        'Birth Year': 1982,
        'Eye Color': 'Hazel' ,
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.33,
        'Weight(lbs)': 115.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Colin Farrell',
        'Sex': 'Male',
        'DOB': '05/31/1976',
        'Birth Year': 1976,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyebrows',
        'Height(ft)': 5.83,
        'Weight(lbs)': 176.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Katie Cassidy',
        'Sex': 'Female',
        'DOB': '11/25/1986',
        'Birth Year': 1986,
        'Eye Color': 'Green',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Lips' ,
        'Height(ft)': 5.50,
        'Weight(lbs)': 119.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Alex Pettyfer',
        'Sex': 'Male',
        'DOB': '04/10/1990',
        'Birth Year': 1990,
        'Eye Color': 'Green',
        'Hair Color': 'Blonde',
        'Distinctive Features': "Body",
        'Height(ft)': 5.91,
        'Weight(lbs)': 174.0, 
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Anna Kendrick',
        'Sex': 'Female',
        'DOB': '08/09/1985',
        'Birth Year': 1985,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.16,
        'Weight(lbs)': 106.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Jeffrey Dean Morgan',
        'Sex': 'Male',
        'DOB': '04/22/1966',
        'Birth Year': 1966,
        'Eye Color': 'Hazel',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Hair',
        'Height(ft)': 6.16,
        'Weight(lbs)': 183.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Jennifer Lawrence',
        'Sex': 'Female',
        'DOB': '08/15/1990',
        'Birth Year': 1990,
        'Eye Color': 'Blue' ,
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.75,
        'Weight(lbs)': 139.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Liam Hemsworth',
        'Sex': 'Male',
        'DOB': '01/13/1990',
        'Birth Year': 1990,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Body',
        'Height(ft)': 6.25,
        'Weight(lbs)': 196.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Emma Stone',
        'Sex': 'Female',
        'DOB': '11/06/1988',
        'Birth Year': 1988,
        'Eye Color': 'Green',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.50, 
        'Weight(lbs)': 115.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Frank Dillane',
        'Sex': 'Male',
        'DOB': '04/21/1991', 
        'Birth Year': 1991,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.91,
        'Weight(lbs)': 163.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Katie McGrath',
        'Sex': 'Female',
        'DOB': '10/24/1983',
        'Birth Year': 1983,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 5.41,
        'Weight(lbs)': 135.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Tom Cruise',
        'Sex': 'Male',
        'DOB': '07/03/1962',
        'Birth Year': 1962,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 5.58,
        'Weight(lbs)': 148.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Sarah Paulson',
        'Sex': 'Female',
        'DOB': '12/17/1974',
        'Birth Year': 1974,
        'Eye Color': 'Brown',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Voice' ,
        'Height(ft)': 5.50,
        'Weight(lbs)': 119.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Keanu Reeves',
        'Sex': 'Male',
        'DOB': '09/02/1964',
        'Birth Year': 1964,
        'Eye Color': 'Hazel',
        'Hair Color': 'Black',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.08,
        'Weight(lbs)': 185.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Cameron Diaz',
        'Sex': 'Female',
        'DOB': '08/30/1972',
        'Birth Year': 1972,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Legs',
        'Height(ft)': 5.75,
        'Weight(lbs)': 128.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'George Clooney',
        'Sex': 'Male',
        'DOB': '05/06/1961',
        'Birth Year': 1961,
        'Eye Color': 'Brown',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.91,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Rachel Weisz',
        'Sex': 'Female',
        'DOB': '03/07/1970',
        'Birth Year': 1970,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.50,
        'Weight(lbs)': 125.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Robert Pattinson',
        'Sex': 'Male',
        'DOB': '05/13/1986',
        'Birth Year': 1986,
        'Eye Color':'Blue',
        'Hair Color':'Brown' ,
        'Distinctive Features':'Eyebrows',
        'Height(ft)': 6.00,
        'Weight(lbs)': 165.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Monica Bellucci',
        'Sex': 'Female',
        'DOB': '09/30/1964',
        'Birth Year': 1964,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Face',
        'Height(ft)': 5.58,
        'Weight(lbs)': 140.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Jake Gyllenhaal',
        'Sex': 'Male',
        'DOB': '12/19/1980',
        'Birth Year': 1980,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.91,
        'Weight(lbs)': 171.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Angela Bassett',
        'Sex': 'Female',
        'DOB': '08/16/1958',
        'Birth Year': 1958,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Smile' ,
        'Height(ft)': 5.25,
        'Weight(lbs)': 134.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Armie Hammer',
        'Sex': 'Male',
        'DOB': '08/28/1986',
        'Birth Year': 1986,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Face',
        'Height(ft)': 6.41,
        'Weight(lbs)': 207.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Marion Cotillard',
        'Sex': 'Female',
        'DOB': '09/30/1975',
        'Birth Year': 1975,
        'Eye Color':'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Forehead',
        'Height(ft)': 5.50,
        'Weight(lbs)': 126.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Idris Elba',
        'Sex': 'Male',
        'DOB': '09/06/1972',
        'Birth Year': 1972,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Voice',
        'Height(ft)': 6.16, 
        'Weight(lbs)': 211.5,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Hailee Steinfeld',
        'Sex': 'Female',
        'DOB': '12/11/1996',
        'Birth Year': 1996,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.50,
        'Weight(lbs)': 130.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'John Stamos',
        'Sex': 'Male',
        'DOB': '08/09/1963',
        'Birth Year': 1963,
        'Eye Color': 'Blue',
        'Hair Color':'Brown',
        'Distinctive Features': 'Smile',
        'Height(ft)': 6.00,
        'Weight(lbs)': 174.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Anna Popplewell',
        'Sex': 'Female',
        'DOB': '12/16/1988',
        'Birth Year': 1988,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.25,
        'Weight(lbs)': 119.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Tyler Blackburn',
        'Sex': 'Male',
        'DOB': '10/12/1986',
        'Birth Year': 1986,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.83,
        'Weight(lbs)': 157.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'AnnaSophia Robb',
        'Sex': 'Female',
        'DOB': '12/08/1993',
        'Birth Year': 1993,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Eyes' ,
        'Height(ft)': 5.00,
        'Weight(lbs)': 108.0, 
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Bradley Cooper',
        'Sex': 'Male',
        'DOB': '01/05/1975',
        'Birth Year': 1975,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.08,
        'Weight(lbs)': 183.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Nicole Kidman',
        'Sex': 'Female',
        'DOB': '06/20/1967',
        'Birth Year': 1967,
        'Eye Color': 'Blue',
        'Hair Color': 'Red',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.91,
        'Weight(lbs)': 140.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Tom Welling',
        'Sex': 'Male',
        'DOB': '04/26/1977',
        'Birth Year': 1977,
        'Eye Color': 'Blue' ,
        'Hair Color': 'Brown',
        'Distinctive Features': 'Jawline' ,
        'Height(ft)': 6.16,
        'Weight(lbs)': 198.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Mercedes Mason',
        'Sex': 'Female',
        'DOB': '01/13/1983',
        'Birth Year': 1983,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.25,
        'Weight(lbs)': 126.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Jesse Williams',
        'Sex': 'Male',
        'DOB': '11/06/1981',
        'Birth Year': 1981,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 5.91, 
        'Weight(lbs)': 175.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Kristen Stewart',
        'Sex': 'Female',
        'DOB': '04/21/1990', 
        'Birth Year': 1990,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.08,
        'Weight(lbs)': 133.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Joe Manganiello',
        'Sex': 'Male',
        'DOB': '10/24/1976',
        'Birth Year': 1976,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 6.08,
        'Weight(lbs)': 205.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Julianne Moore',
        'Sex': 'Female',
        'DOB': '07/03/1960',
        'Birth Year': 1960,
        'Eye Color': 'Blue',
        'Hair Color': 'Red',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.25,
        'Weight(lbs)': 128.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'John Boyega',
        'Sex': 'Male',
        'DOB': '12/17/1992',
        'Birth Year': 1992,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 5.50,
        'Weight(lbs)': 179.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 0
    },

    {
        'Name': 'Kate Winslet',
        'Sex': 'Female',
        'DOB': '10/05/1975',
        'Birth Year': 1975,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.50,
        'Weight(lbs)': 139.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Garrett Hedlund',
        'Sex': 'Male',
        'DOB': '09/03/1984',
        'Birth Year': 1984,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.16,
        'Weight(lbs)': 201.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'No',
        'Hot Test': 0
    },

    {
        'Name': 'Jennifer Aniston',
        'Sex': 'Female',
        'DOB': '02/11/1969',
        'Birth Year': 1969,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.33,
        'Weight(lbs)': 117.0,
        'Zodiac Sign': 'Aquarius',
        'Tattoo Body Art':'No',
        'Hot Test': 0
    },

    {
        'Name': 'Benedict Cumberbatch',
        'Sex': 'Male',
        'DOB': '07/19/1976',
        'Birth Year': 1976,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Cheekbones',
        'Height(ft)': 6.00,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Bonnie Aarons',
        'Sex': 'Female',
        'DOB': '06/03/1979',
        'Birth Year': 1979,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Nose',
        'Height(ft)': 5.58,
        'Weight(lbs)': 123.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Barkhad Abdi',
        'Sex': 'Male',
        'DOB': '04/10/1985', 
        'Birth Year': 1985,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Goatee',
        'Height(ft)': 5.83,
        'Weight(lbs)': 143.3,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Woody Allen',
        'Sex': 'Male',
        'DOB': '12/01/1935',
        'Birth Year': 1935,
        'Eye Color': 'Brown',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Jawline',
        'Height(ft)': 5.41,
        'Weight(lbs)': 147.5,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Devon Aoki',
        'Sex': 'Female',
        'DOB': '08/10/1982',
        'Birth Year': 1982,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Body',
        'Height(ft)': 5.41,
        'Weight(lbs)': 110.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Rowan Atkinson',
        'Sex': 'Male',
        'DOB': '01/06/1955',
        'Birth Year': 1955,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyebrows',
        'Height(ft)': 5.91,
        'Weight(lbs)': 178,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Jonathan Banks',
        'Sex': 'Male',
        'DOB': '01/31/1947',
        'Birth Year': 1947,
        'Eye Color': 'Green',
        'Hair Color': 'Bald',
        'Distinctive Features': 'Ears',
        'Height(ft)': 5.75,
        'Weight(lbs)': 171.9,
        'Zodiac Sign': 'Aquarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Peter Berg',
        'Sex': 'Male',
        'DOB': '03/11/1964',
        'Birth Year': 1964,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Nose',
        'Height(ft)': 6.08, 
        'Weight(lbs)': 127.8,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Michael Berryman',
        'Sex': 'Male',
        'DOB': '09/04/1948',
        'Birth Year': 1948,
        'Eye Color': 'Green',
        'Hair Color': 'Bald',
        'Distinctive Features': 'Ears',
        'Height(ft)': 6.16,
        'Weight(lbs)': 152.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Sandra Bernhard',
        'Sex': 'Female',
        'DOB': '06/06/1955',
        'Birth Year': 1955,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Jawline',
        'Height(ft)': 5.08,
        'Weight(lbs)': 143.5,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Susan Boyle',
        'Sex': 'Female',
        'DOB': '04/01/1961',
        'Birth Year': 1961,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Body',
        'Height(ft)': 5.33,
        'Weight(lbs)': 150.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Jo Brand',
        'Sex': 'Female',
        'DOB': '07/23/1957',
        'Birth Year': 1957,
        'Eye Color': 'Brown',
        'Hair Color': 'Red',
        'Distinctive Features': 'Nose',
        'Height(ft)': 5.33,
        'Weight(lbs)': 154.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Adrien Brody',
        'Sex': 'Male',
        'DOB': '04/14/1973',
        'Birth Year': 1973,
        'Eye Color': 'Hazel',
        'Hair Color': 'Black',
        'Distinctive Features': 'Nose',
        'Height(ft)': 6.25,
        'Weight(lbs)': 161.0, 
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Steve Buscemi',
        'Sex': 'Male',
        'DOB': '12/13/1957',
        'Birth Year': 1957,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 5.75,
        'Weight(lbs)': 154.5,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Gary Busey',
        'Sex': 'Male',
        'DOB': '06/29/1944',
        'Birth Year': 1944,
        'Eye Color': 'Green',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.00,
        'Weight(lbs)': 240.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Vincent Cassel',
        'Sex': 'Male',
        'DOB': '11/23/1966',
        'Birth Year': 1966,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Jawline',
        'Height(ft)': 6.08,
        'Weight(lbs)': 176.5,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Michael Cera',
        'Sex': 'Male',
        'DOB': '06/07/1988',
        'Birth Year': 1988,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.75,
        'Weight(lbs)': 154.5,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Glenn Close',
        'Sex': 'Female',
        'DOB': '03/19/1947',
        'Birth Year': 1947,
        'Eye Color': 'Green',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Chin',
        'Height(ft)': 5.41, 
        'Weight(lbs)': 128.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Toni Collette',
        'Sex': 'Female',
        'DOB': '11/01/1972', 
        'Birth Year': 1972,
        'Eye Color': 'Green',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Nose',
        'Height(ft)': 5.66,
        'Weight(lbs)': 130.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Macaulay Culkin',
        'Sex': 'Male',
        'DOB': '08/26/1980',
        'Birth Year': 1980,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.58,
        'Weight(lbs)': 137.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Joan Cusack',
        'Sex': 'Female',
        'DOB': '10/11/1962',
        'Birth Year': 1962,
        'Eye Color': 'Blue',
        'Hair Color': 'Red',
        'Distinctive Features': 'Nose',
        'Height(ft)': 5.75,
        'Weight(lbs)': 150.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Miley Cyrus',
        'Sex': 'Female',
        'DOB': '11/23/1992',
        'Birth Year': 1992,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Ears',
        'Height(ft)': 5.41,
        'Weight(lbs)': 106.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Willem Dafoe',
        'Sex': 'Male',
        'DOB': '07/22/1955',
        'Birth Year': 1955,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Cheekbones',
        'Height(ft)': 5.75,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Gerard Depardieu',
        'Sex': 'Male',
        'DOB': '12/27/1948',
        'Birth Year': 1948,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Nose',
        'Height(ft)': 5.91,
        'Weight(lbs)': 209.5,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Illeana Douglas',
        'Sex': 'Female',
        'DOB': '07/25/1965',
        'Birth Year': 1965,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.75,
        'Weight(lbs)': 137.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Rachel Dratch',
        'Sex': 'Female',
        'DOB': '02/22/1966',
        'Birth Year': 1966,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.00,
        'Weight(lbs)': 123.5,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Adam Driver',
        'Sex': 'Male',
        'DOB': '11/19/1983',
        'Birth Year': 1983,
        'Eye Color': 'Hazel',
        'Hair Color': 'Black',
        'Distinctive Features': 'Ears',
        'Height(ft)': 6.25,
        'Weight(lbs)': 194.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Anne-Marie Duff',
        'Sex': 'Female',
        'DOB': '10/08/1970',
        'Birth Year': 1970,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.58,
        'Weight(lbs)': 119.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Lena Dunham',
        'Sex': 'Female',
        'DOB': '05/13/1986',
        'Birth Year': 1986,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.25,
        'Weight(lbs)': 145.5,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Shelley Duvall',
        'Sex': 'Female',
        'DOB': '07/07/1949',
        'Birth Year': 1949,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 5.66,
        'Weight(lbs)': 119.3,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Chris Elliott',
        'Sex': 'Male',
        'DOB': '05/31/1960',
        'Birth Year': 1960,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Bald',
        'Height(ft)': 6.00,
        'Weight(lbs)': 192.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Flavor Flav',
        'Sex': 'Male',
        'DOB': '04/16/1959',
        'Birth Year': 1959,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 5.58,
        'Weight(lbs)': 154.5,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },

    {
        'Name': 'Flea',
        'Sex': 'Male',
        'DOB': '10/16/1962',
        'Birth Year': 1962,
        'Eye Color': 'Blue',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Stubble',
        'Height(ft)': 5.50,
        'Weight(lbs)': 154.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Brendan Fraser',
        'Sex': 'Male',
        'DOB': '12/03/1968',
        'Birth Year': 1968,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Voice',
        'Height(ft)': 6.25,
        'Weight(lbs)': 209.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Joseph Gatt',
        'Sex': 'Male',
        'DOB': '12/03/1974',
        'Birth Year': 1974,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Bald',
        'Height(ft)': 6.08,
        'Weight(lbs)': 168.8,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Paul Giamatti',
        'Sex': 'Male',
        'DOB': '06/06/1967',
        'Birth Year': 1967,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Cheekbones',
        'Height(ft)': 5.75,
        'Weight(lbs)': 168.8,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Crispin Glover',
        'Sex': 'Male',
        'DOB': '04/20/1964',
        'Birth Year': 1964,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Nose',
        'Height(ft)': 5.08,
        'Weight(lbs)': 168.8,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Adam Godley',
        'Sex': 'Male',
        'DOB': '07/22/1964',
        'Birth Year': 1964,
        'Eye Color': 'Hazel',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.08,
        'Weight(lbs)': 177.8,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Gilbert Gottfried',
        'Sex': 'Male',
        'DOB': '02/28/1955',
        'Birth Year': 1955,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Cheekbones',
        'Height(ft)': 5.25,
        'Weight(lbs)': 141.8,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Nancy Grace',
        'Sex': 'Female',
        'DOB': '10/23/1959',
        'Birth Year': 1959,
        'Eye Color': 'Brown',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.16,
        'Weight(lbs)': 112.5,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Kathy Griffin',
        'Sex': 'Female',
        'DOB': '11/04/1960',
        'Birth Year': 1960,
        'Eye Color': 'Green',
        'Hair Color': 'Red',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.33,
        'Weight(lbs)': 121.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Melanie Griffith',
        'Sex': 'Female',
        'DOB': '08/09/1957',
        'Birth Year': 1957,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Body',
        'Height(ft)': 5.75,
        'Weight(lbs)': 139.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Luis Guzmán',
        'Sex': 'Male',
        'DOB': '08/28/1956',
        'Birth Year': 1956,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Forehead',
        'Height(ft)': 5.41,
        'Weight(lbs)': 168.8,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },

    {
        'Name': 'Maggie Gyllenhaal',
        'Sex': 'Female',
        'DOB': '11/16/1977',
        'Birth Year': 1977,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.75,
        'Weight(lbs)': 132.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Jackie Earle Haley',
        'Sex': 'Male',
        'DOB': '12/03/1968',
        'Birth Year': 1968,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Jawline',
        'Height(ft)': 5.50,
        'Weight(lbs)': 143.5,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Jared Harris',
        'Sex': 'Male',
        'DOB': '08/24/1961',
        'Birth Year': 1961,
        'Eye Color': 'Blue',
        'Hair Color': 'Red',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.91,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Lucas Hedges',
        'Sex': 'Male',
        'DOB': '12/12/1996',
        'Birth Year': 1996,
        'Eye Color': 'Blue',
        'Hair Color': 'Red',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.91,
        'Weight(lbs)': 154.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Simon Helberg',
        'Sex': 'Male',
        'DOB': '12/09/1980',
        'Birth Year': 1980,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Voice',
        'Height(ft)': 5.58,
        'Weight(lbs)': 154.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Jonah Hill',
        'Sex': 'Male',
        'DOB': '12/20/1983',
        'Birth Year': 1983,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Chin',
        'Height(ft)': 5.58,
        'Weight(lbs)': 192.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Perez Hilton',
        'Sex': 'Male',
        'DOB': '03/23/1978',
        'Birth Year': 1978,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 6.00,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Clint Howard',
        'Sex': 'Male',
        'DOB': '4/20/1959',
        'Birth Year': 1959,
        'Eye Color': 'Blue',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 5.58,
        'Weight(lbs)': 165.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Linda Hunt',
        'Sex': 'Female',
        'DOB': '04/02/1945',
        'Birth Year': 1945,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Height',
        'Height(ft)': 4.75,
        'Weight(lbs)': 143.0,
        'Zodiac Sign': 'Aquarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Mick Jagger',
        'Sex': 'Male',
        'DOB': '07/26/1943', 
        'Birth Year': 1943,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.75,
        'Weight(lbs)': 161.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Jay-Z',
        'Sex': 'Male',
        'DOB': '12/04/1969',
        'Birth Year': 1969,
        'Eye Color': 'Black',
        'Hair Color': 'Black',
        'Distinctive Features': 'Lips',
        'Height(ft)': 6.16,
        'Weight(lbs)': 183.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Caitlyn Jenner',
        'Sex': 'Female',
        'DOB': '10/28/1949',
        'Birth Year': 1949,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 6.16,
        'Weight(lbs)': 194.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Caleb Landry Jones',
        'Sex': 'Male',
        'DOB': '12/07/1989',
        'Birth Year': 1989,
        'Eye Color': 'Blue',
        'Hair Color': 'Red',
        'Distinctive Features': 'Cheekbones',
        'Height(ft)': 5.91,
        'Weight(lbs)': 143.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Kelis',
        'Sex': 'Female',
        'DOB': '08/21/1979',
        'Birth Year': 1979,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.75,
        'Weight(lbs)': 139.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Barry Keoghan',
        'Sex': 'Male',
        'DOB': '10/18/1992',
        'Birth Year': 1992,
        'Eye Color': 'Blue',
        'Hair Color': 'Black',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.50, 
        'Weight(lbs)': 159.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Wiz Khalifa',
        'Sex': 'Male',
        'DOB': '09/08/1987',
        'Birth Year': 1987,
        'Eye Color': 'Brown',
        'Hair Color': 'Black',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 6.33,
        'Weight(lbs)': 172,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Salman Khan',
        'Sex': 'Male',
        'DOB': '12/27/1965',
        'Birth Year': 1965,
        'Eye Color': 'Brown' ,
        'Hair Color': 'Black',
        'Distinctive Features': 'Muscular',
        'Height(ft)': 5.66,
        'Weight(lbs)': 194,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Stephen King',
        'Sex': 'Male',
        'DOB': '09/21/1947',
        'Birth Year': 1947,
        'Eye Color': 'Blue',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Lips',
        'Height(ft)': 6.33,
        'Weight(lbs)': 165.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Lisa Lampanelli',
        'Sex': 'Female',
        'DOB': '07/19/1961',
        'Birth Year': 1961,
        'Eye Color': 'Brown',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Chin' ,
        'Height(ft)': 5.58,
        'Weight(lbs)': 141.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Tomas Lemarquis',
        'Sex': 'Male',
        'DOB': '08/03/1977',
        'Birth Year': 1977,
        'Eye Color': 'Blue',
        'Hair Color': 'Bald',
        'Distinctive Features': "Skin",
        'Height(ft)': 5.75,
        'Weight(lbs)': 174.0, 
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Amanda Lepore',
        'Sex': 'Female',
        'DOB': '11/21/1967',
        'Birth Year': 1967,
        'Eye Color': 'Brown',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Lips',
        'Height(ft)': 5.16,
        'Weight(lbs)': 110.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Lorde',
        'Sex': 'Female',
        'DOB': '11/07/1996',
        'Birth Year': 1996,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.41,
        'Weight(lbs)': 134.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Lyle Lovett',
        'Sex': 'Male',
        'DOB': '11/1/1957',
        'Birth Year': 1957,
        'Eye Color': 'Brown' ,
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Lips',
        'Height(ft)': 6.00,
        'Weight(lbs)': 171.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Matt Lucas',
        'Sex': 'Male',
        'DOB': '05/05/1974',
        'Birth Year': 1974,
        'Eye Color': 'Hazel',
        'Hair Color': 'Bald',
        'Distinctive Features': 'Skin',
        'Height(ft)': 5.50,
        'Weight(lbs)': 185.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Shane Macgowan',
        'Sex': 'Male',
        'DOB': '12/25/1957',
        'Birth Year': 1957,
        'Eye Color': 'Blue',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 5.75, 
        'Weight(lbs)': 115.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'William H Macy',
        'Sex': 'Male',
        'DOB': '03/13/1950', 
        'Birth Year': 1950,
        'Eye Color': 'Green',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.66,
        'Weight(lbs)': 159.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'John Malkovich',
        'Sex': 'Male',
        'DOB': '12/9/1953',
        'Birth Year': 1953,
        'Eye Color': 'Hazel',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Chin',
        'Height(ft)': 6.00,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Marilyn Manson',
        'Sex': 'Male',
        'DOB': '01/05/1969',
        'Birth Year': 1969,
        'Eye Color': 'Hazel',
        'Hair Color': 'Black',
        'Distinctive Features': 'Lips',
        'Height(ft)': 6.08,
        'Weight(lbs)': 181.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Biz Markie',
        'Sex': 'Male',
        'DOB': '04/08/1964',
        'Birth Year': 1964,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 6.25,
        'Weight(lbs)': 220.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art': 'Yes',
        'Hot Test': 1
    },

    {
        'Name': 'Eddie Marsan',
        'Sex': 'Male',
        'DOB': '06/09/1968',
        'Birth Year': 1968,
        'Eye Color': 'Green',
        'Hair Color': 'Gray',
        'Distinctive Features': 'Cheeks',
        'Height(ft)': 5.66,
        'Weight(lbs)': 150.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Heather Matarazzo',
        'Sex': 'Female',
        'DOB': '11/10/1982',
        'Birth Year': 1982,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Teeth',
        'Height(ft)': 5.33,
        'Weight(lbs)': 121.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name':  'Jack McBrayer',
        'Sex': 'Male',
        'DOB': '05/27/1973',
        'Birth Year': 1973,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.83,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art': 'No',
        'Hot Test': 1
    },

    {
        'Name': 'Jack McBrayer',
        'Sex': 'Male',
        'DOB': '7/19/1976',
        'Birth Year': 1976,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.00,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Ozzy Osbourne',
        'Sex': 'Male',
        'DOB': '6/3/1953',
        'Birth Year': 1953,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.33,
        'Weight(lbs)': 132.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art':'Yes',
        'Hot Test': 1
    },
    
    {
        'Name': 'Kate Micucci',
        'Sex': 'Female',
        'DOB': '4/10/1983',
        'Birth Year': 1983,
        'Eye Color': 'Black',
        'Hair Color': 'Black',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.33,
        'Weight(lbs)': 145.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Haley Joel Osment',
        'Sex': 'Male',
        'DOB': '12/1/1985',
        'Birth Year': 1985,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.33,
        'Weight(lbs)': 148.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Sarah Jessica Parker',
        'Sex': 'Female',
        'DOB': '8/10/1965',
        'Birth Year': 1965,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.41,
        'Weight(lbs)': 125.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Camilla Parker-Bowles',
        'Sex': 'Female',
        'DOB': '6/6/1947',
        'Birth Year': 1947,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.41,
        'Weight(lbs)': 146.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Pedro Pascal',
        'Sex': 'Male',
        'DOB': '1/31/1975',
        'Birth Year': 1975,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.66,
        'Weight(lbs)': 165.0,
        'Zodiac Sign': 'Aquarius',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Logan Paul',
        'Sex': 'Male',
        'DOB': '3/11/1994',
        'Birth Year': 1994,
        'Eye Color': 'Blue',
        'Hair Color': 'BLonde',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.00,
        'Weight(lbs)': 185.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Ron Perlman',
        'Sex': 'Male',
        'DOB': '9/4/1953',
        'Birth Year': 1953,
        'Eye Color': 'Blue',
        'Hair Color': 'White',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.08,
        'Weight(lbs)': 175.0,
        'Zodiac Sign': 'Leo',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Michael Phelps',
        'Sex': 'Male',
        'DOB': '6/6/1965',
        'Birth Year': 1965,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Body',
        'Height(ft)': 6.08,
        'Weight(lbs)': 155.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art':'Yes',
        'Hot Test': 1
    },
    
    {
        'Name': 'Jesse Plemons',
        'Sex': 'Male',
        'DOB': '4/1/1988',
        'Birth Year': 1988,
        'Eye Color': 'Brown',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.08,
        'Weight(lbs)': 160.0,
        'Zodiac Sign': 'Taurus',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Iggy Pop',
        'Sex': 'Male',
        'DOB': '7/23/1951',
        'Birth Year': 1951,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.00,
        'Weight(lbs)': 172.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Randy Quaid',
        'Sex': 'Male',
        'DOB': '4/13/1957',
        'Birth Year': 1957,
        'Eye Color': 'Blue',
        'Hair Color': 'Red',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.08,
        'Weight(lbs)': 225.0,
        'Zodiac Sign': 'Aries',
        'Tattoo Body Art':'Yes',
        'Hot Test': 1
    },
    
    {
        'Name': 'John C. Reilly',
        'Sex': 'Male',
        'DOB': '12/13/1965',
        'Birth Year': 1965,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.08,
        'Weight(lbs)': 190.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Giovanni Ribisi',
        'Sex': 'Male',
        'DOB': '6/29/1974',
        'Birth Year': 1974,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.58,
        'Weight(lbs)': 160.0,
        'Zodiac Sign': 'Capricorn',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Kid Rock',
        'Sex': 'Male',
        'DOB': '11/23/1966',
        'Birth Year': 1966,
        'Eye Color': 'Blue',
        'Hair Color': 'Blonde',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.08,
        'Weight(lbs)': 177.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Dennis Rodman',
        'Sex': 'Male',
        'DOB': '7/7/1961',
        'Birth Year': 1961,
        'Eye Color': 'Black',
        'Hair Color': 'Black',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 6.25,
        'Weight(lbs)': 220.0,
        'Zodiac Sign': 'Gemini',
        'Tattoo Body Art':'Yes',
        'Hot Test': 1
    },
    
    {
        'Name': 'Mickey Rourke',
        'Sex': 'Male',
        'DOB': '3/19/1947',
        'Birth Year': 1947,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.08,
        'Weight(lbs)': 198.0,
        'Zodiac Sign': 'Pisces',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Geoffrey Rush',
        'Sex': 'Male',
        'DOB': '11/1/1942',
        'Birth Year': 1942,
        'Eye Color': 'Blue',
        'Hair Color': 'White',
        'Distinctive Features': 'Face',
        'Height(ft)': 5.08,
        'Weight(lbs)': 150.0,
        'Zodiac Sign': 'Scorpio',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Paul Rust',
        'Sex': 'Male',
        'DOB': '8/26/1980',
        'Birth Year': 1980,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.58,
        'Weight(lbs)': 155.0,
        'Zodiac Sign': 'Virgo',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Kristen Schaal',
        'Sex': 'Female',
        'DOB': '10/11/1978',
        'Birth Year': 1978,
        'Eye Color': 'Blue',
        'Hair Color': 'Red',
        'Distinctive Features': 'Eyes',
        'Height(ft)': 5.50,
        'Weight(lbs)': 140.0,
        'Zodiac Sign': 'Libra',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Steven Seagal',
        'Sex': 'Male',
        'DOB': '11/23/1952',
        'Birth Year': 1952,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.91,
        'Weight(lbs)': 215.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Seal',
        'Sex': 'Male',
        'DOB': '7/22/1963',
        'Birth Year': 1963,
        'Eye Color': 'Black',
        'Hair Color': 'Bald',
        'Distinctive Features': 'Face',
        'Height(ft)': 5.08,
        'Weight(lbs)': 165.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Charlie Sheen',
        'Sex': 'Male',
        'DOB': '12/27/1965',
        'Birth Year': 1965,
        'Eye Color': 'Brown',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 6.00,
        'Weight(lbs)': 180.0,
        'Zodiac Sign': 'Sagittarius',
        'Tattoo Body Art':'Yes',
        'Hot Test': 1
    },
    
    {
        'Name': 'Gabourey Sidibe',
        'Sex': 'Female',
        'DOB': '7/25/1983',
        'Birth Year': 1983,
        'Eye Color': 'Black',
        'Hair Color': 'Black',
        'Distinctive Features': 'Smile',
        'Height(ft)': 5.08,
        'Weight(lbs)': 290.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
    {
        'Name': 'Matt Smith',
        'Sex': 'Male',
        'DOB': '7/19/1982',
        'Birth Year': 1982,
        'Eye Color': 'Blue',
        'Hair Color': 'Brown',
        'Distinctive Features': 'Hair',
        'Height(ft)': 5.75,
        'Weight(lbs)': 159.0,
        'Zodiac Sign': 'Cancer',
        'Tattoo Body Art':'No',
        'Hot Test': 1
    },
    
]

celeb_df = pd.DataFrame.from_dict(celeb_dict)  
celeb_df.tail()

,Name,Sex,DOB,Birth Year,Eye Color,Hair Color,Distinctive Features,Height(ft),Weight(lbs),Zodiac Sign,Tattoo Body Art,Hot Test
196,Steven Seagal,Male,11/23/1952,1952,Brown,Brown,Smile,5.91,215.0,Sagittarius,No,1
197,Seal,Male,7/22/1963,1963,Black,Bald,Face,5.08,165.0,Cancer,No,1
198,Charlie Sheen,Male,12/27/1965,1965,Brown,Brown,Hair,6.00,180.0,Sagittarius,Yes,1
199,Gabourey Sidibe,Female,7/25/1983,1983,Black,Black,Smile,5.08,290.0,Cancer,No,1
200,Matt Smith,Male,7/19/1982,1982,Blue,Brown,Hair,5.75,159.0,Cancer,No,1


In [11]:
# add Age column and a column for height/weight ratio to be used in our model.

celeb_df['Ht/Wt Ratio'] = round(celeb_df['Weight(lbs)']/celeb_df['Height(ft)'], 2)
celeb_df['Age'] = 2020-celeb_df['Birth Year']
celeb_df.tail()

,Name,Sex,DOB,Birth Year,Eye Color,Hair Color,Distinctive Features,Height(ft),Weight(lbs),Zodiac Sign,Tattoo Body Art,Hot Test,Ht/Wt Ratio,Age
196,Steven Seagal,Male,11/23/1952,1952,Brown,Brown,Smile,5.91,215.0,Sagittarius,No,1,36.38,68
197,Seal,Male,7/22/1963,1963,Black,Bald,Face,5.08,165.0,Cancer,No,1,32.48,57
198,Charlie Sheen,Male,12/27/1965,1965,Brown,Brown,Hair,6.00,180.0,Sagittarius,Yes,1,30.00,55
199,Gabourey Sidibe,Female,7/25/1983,1983,Black,Black,Smile,5.08,290.0,Cancer,No,1,57.09,37
200,Matt Smith,Male,7/19/1982,1982,Blue,Brown,Hair,5.75,159.0,Cancer,No,1,27.65,38


In [13]:
# Build our sqlite database 
# create our hot table to hold our data

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def drop_table(conn, drop_table_sql):
    """ drop a table from the drop_table_sql statement
    :param conn: Connection object
    :param drop_table_sql: a drop TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(drop_table_sql)
    except Error as e:
        print(e)
def main():
    database = r"../DataBase/Are_You_Hot.db"
    sql_create_hot_table = """CREATE TABLE hot(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Name text NOT NULL,
                                        Sex text,
                                        DOB text,
                                        Birth Year int,
                                        Eye Color text,
                                        Hair Color text,
                                        Distinctive Features text,
                                        Height(ft) float,
                                        Weight(lbs) float,
                                        Zodiac Sign text,
                                        Tattoo Body Art text,
                                        Hot Test int,
                                        Ht/Wt Ratio float,
                                        Age int
                                    ); """										
    sql_drop_hot_table = """DROP TABLE if exists hot;"""

    # create a database connection
    conn = create_connection(database)
    # create tables
    if conn is not None:
        # create hot table
        drop_table(conn, sql_drop_hot_table)
        create_table(conn, sql_create_hot_table)
        

    else:
        print("Error! cannot create the database connection.")
if __name__ == '__main__':
    main()

near "(": syntax error


In [14]:
# create engine

engine = create_engine('sqlite:///../DataBase/Are_You_Hot.db')

# populate our hot table with data from our dataframe

celeb_df.to_sql(name='hot', con=engine, if_exists = "append", index=False)

In [15]:
# query our table to make sure database is working

hot_test = pd.read_sql('select * from hot', engine)
hot_test.tail()

,Name,Sex,DOB,Birth Year,Eye Color,Hair Color,Distinctive Features,Height(ft),Weight(lbs),Zodiac Sign,Tattoo Body Art,Hot Test,Ht/Wt Ratio,Age
196,Steven Seagal,Male,11/23/1952,1952,Brown,Brown,Smile,5.91,215.0,Sagittarius,No,1,36.38,68
197,Seal,Male,7/22/1963,1963,Black,Bald,Face,5.08,165.0,Cancer,No,1,32.48,57
198,Charlie Sheen,Male,12/27/1965,1965,Brown,Brown,Hair,6.00,180.0,Sagittarius,Yes,1,30.00,55
199,Gabourey Sidibe,Female,7/25/1983,1983,Black,Black,Smile,5.08,290.0,Cancer,No,1,57.09,37
200,Matt Smith,Male,7/19/1982,1982,Blue,Brown,Hair,5.75,159.0,Cancer,No,1,27.65,38
